# Post-Scraping Data Processing

This notebook performs data cleaning and standardization on **BOTH** databases after scraping.

**Databases Updated:**
- `projections.db` - Projections table
- `league.db` - NFL Players table

**Processing Steps:**
1. Standardize team names (WSH → WAS, JAC → JAX, LA → LAR)
2. Standardize defense positions (D/ST → DST, DEF → DST)
3. Convert FB positions to RB
4. Remove defensive players (DB, LB, DL, NT)
5. Fix specific player positions (Travis Hunter → WR)

**Run this after scraping to ensure clean, standardized data across both databases.**


In [ ]:
import sqlite3
import sys
import pandas as pd
from pathlib import Path

# Add parent directory to path for imports
sys.path.append(str(Path().absolute().parent.parent))
sys.path.append(str(Path().absolute().parent / 'scrapers'))

from backend.scrapers.database import ProjectionsDB
from backend.scrapers.database_league import LeagueDB


## 1. Connect to Database and Check Current State


In [ ]:
# Connect to both databases
conn_proj = sqlite3.connect('../data/databases/projections.db')
cursor_proj = conn_proj.cursor()

conn_league = sqlite3.connect('../data/databases/league.db')
cursor_league = conn_league.cursor()

print("="*70)
print("📊 PROJECTIONS DATABASE (projections.db)")
print("="*70)

# Check total records before processing
cursor_proj.execute("SELECT COUNT(*) FROM projections")
total_proj_before = cursor_proj.fetchone()[0]
print(f"Total records: {total_proj_before}")

# Check current team names
cursor_proj.execute("SELECT DISTINCT team FROM projections ORDER BY team")
teams_proj = [row[0] for row in cursor_proj.fetchall()]
print(f"\n🏈 Unique teams ({len(teams_proj)}): {teams_proj}")

# Check current positions
cursor_proj.execute("SELECT position, COUNT(*) FROM projections GROUP BY position ORDER BY COUNT(*) DESC")
positions_proj = cursor_proj.fetchall()
print("\n📋 Positions breakdown:")
for pos, count in positions_proj:
    print(f"   {pos}: {count}")

print("\n" + "="*70)
print("🏈 SLEEPER NFL PLAYERS DATABASE (league.db)")
print("="*70)

# Check total records before processing
cursor_league.execute("SELECT COUNT(*) FROM nfl_players")
total_league_before = cursor_league.fetchone()[0]
print(f"Total NFL players: {total_league_before}")

# Check current team names
cursor_league.execute("SELECT DISTINCT team FROM nfl_players ORDER BY team")
teams_league = [row[0] for row in cursor_league.fetchall()]
print(f"\n🏈 Unique teams ({len(teams_league)}): {teams_league}")

# Check current positions
cursor_league.execute("SELECT position, COUNT(*) FROM nfl_players GROUP BY position ORDER BY COUNT(*) DESC")
positions_league = cursor_league.fetchall()
print("\n📋 Positions breakdown:")
for pos, count in positions_league[:15]:  # Show top 15
    print(f"   {pos}: {count}")


📊 PROJECTIONS DATABASE (projections.db)
Total records: 1313

🏈 Unique teams (32): ['ARI', 'ATL', 'BAL', 'BUF', 'CAR', 'CHI', 'CLE', 'DEN', 'DET', 'FA', 'GB', 'HOU', 'IND', 'JAC', 'JAX', 'LA', 'LAC', 'LAR', 'LV', 'MIA', 'MIN', 'NE', 'NO', 'NYG', 'NYJ', 'PHI', 'PIT', 'SEA', 'SF', 'TB', 'WAS', 'WSH']

📋 Positions breakdown:
   WR: 403
   RB: 281
   TE: 259
   QB: 179
   K: 82
   DL: 50
   D/ST: 28
   DST: 25
   FB: 4
   NT: 2

🏈 SLEEPER NFL PLAYERS DATABASE (league.db)
Total NFL players: 3968

🏈 Unique teams (33): [None, 'ARI', 'ATL', 'BAL', 'BUF', 'CAR', 'CHI', 'CIN', 'CLE', 'DAL', 'DEN', 'DET', 'GB', 'HOU', 'IND', 'JAX', 'KC', 'LAC', 'LAR', 'LV', 'MIA', 'MIN', 'NE', 'NO', 'NYG', 'NYJ', 'PHI', 'PIT', 'SEA', 'SF', 'TB', 'TEN', 'WAS']

📋 Positions breakdown:
   WR: 1663
   RB: 866
   TE: 782
   QB: 445
   K: 180
   DST: 32


## 2. Standardize Team Names: WSH → WAS


In [ ]:
# Update WSH to WAS in PROJECTIONS
cursor_proj.execute("""
    UPDATE projections 
    SET team = 'WAS', updated_at = CURRENT_TIMESTAMP 
    WHERE team = 'WSH'
""")
wsh_proj_count = cursor_proj.rowcount
conn_proj.commit()

# Update WSH to WAS in NFL PLAYERS
cursor_league.execute("""
    UPDATE nfl_players 
    SET team = 'WAS', updated_at = CURRENT_TIMESTAMP 
    WHERE team = 'WSH'
""")
wsh_league_count = cursor_league.rowcount
conn_league.commit()

print(f"✅ Changed WSH → WAS:")
print(f"   Projections DB: {wsh_proj_count} records updated")
print(f"   NFL Players DB: {wsh_league_count} records updated")


✅ Changed WSH → WAS:
   Projections DB: 26 records updated
   NFL Players DB: 0 records updated


## 3. Standardize Team Names: JAC → JAX


In [4]:
# Update JAC to JAX in PROJECTIONS
cursor_proj.execute("""
    UPDATE projections 
    SET team = 'JAX', updated_at = CURRENT_TIMESTAMP 
    WHERE team = 'JAC'
""")
jac_proj_count = cursor_proj.rowcount
conn_proj.commit()

# Update JAC to JAX in NFL PLAYERS
cursor_league.execute("""
    UPDATE nfl_players 
    SET team = 'JAX', updated_at = CURRENT_TIMESTAMP 
    WHERE team = 'JAC'
""")
jac_league_count = cursor_league.rowcount
conn_league.commit()

print(f"✅ Changed JAC → JAX:")
print(f"   Projections DB: {jac_proj_count} records updated")
print(f"   NFL Players DB: {jac_league_count} records updated")


✅ Changed JAC → JAX:
   Projections DB: 19 records updated
   NFL Players DB: 0 records updated


## 5. Standardize Defense Positions: D/ST & DEF → DST


In [5]:
# Update D/ST to DST in PROJECTIONS
cursor_proj.execute("""
    UPDATE projections 
    SET position = 'DST', updated_at = CURRENT_TIMESTAMP 
    WHERE position = 'D/ST'
""")
dst_proj_count = cursor_proj.rowcount
conn_proj.commit()

# Update DEF to DST in NFL PLAYERS
cursor_league.execute("""
    UPDATE nfl_players 
    SET position = 'DST', updated_at = CURRENT_TIMESTAMP 
    WHERE position = 'DEF'
""")
def_league_count = cursor_league.rowcount
conn_league.commit()

print(f"✅ Changed D/ST → DST:")
print(f"   Projections DB: {dst_proj_count} records updated")
print(f"✅ Changed DEF → DST:")
print(f"   NFL Players DB: {def_league_count} records updated")


✅ Changed D/ST → DST:
   Projections DB: 28 records updated
✅ Changed DEF → DST:
   NFL Players DB: 0 records updated


## 4. Standardize Team Names: LA → LAR


In [6]:
# Update LA to LAR in PROJECTIONS
cursor_proj.execute("""
    UPDATE projections 
    SET team = 'LAR', updated_at = CURRENT_TIMESTAMP 
    WHERE team = 'LA'
""")
la_proj_count = cursor_proj.rowcount
conn_proj.commit()

# Update LA to LAR in NFL PLAYERS
cursor_league.execute("""
    UPDATE nfl_players 
    SET team = 'LAR', updated_at = CURRENT_TIMESTAMP 
    WHERE team = 'LA'
""")
la_league_count = cursor_league.rowcount
conn_league.commit()

print(f"✅ Changed LA → LAR:")
print(f"   Projections DB: {la_proj_count} records updated")
print(f"   NFL Players DB: {la_league_count} records updated")


✅ Changed LA → LAR:
   Projections DB: 15 records updated
   NFL Players DB: 0 records updated


## 6. Convert FB Positions to RB


In [7]:
# Show FB players before conversion in PROJECTIONS
cursor_proj.execute("""
    SELECT player_first_name, player_last_name, position, team, source_website 
    FROM projections 
    WHERE position = 'FB'
""")
fb_proj_players = cursor_proj.fetchall()

if fb_proj_players:
    print(f"🏃 Found {len(fb_proj_players)} FB players in Projections DB:")
    for first, last, pos, team, source in fb_proj_players:
        print(f"   - {first} {last} ({team}) from {source}")
else:
    print("ℹ️  No FB players found in Projections DB")

# Show FB players in NFL PLAYERS
cursor_league.execute("""
    SELECT first_name, last_name, position, team 
    FROM nfl_players 
    WHERE position = 'FB'
""")
fb_league_players = cursor_league.fetchall()

if fb_league_players:
    print(f"\n🏃 Found {len(fb_league_players)} FB players in NFL Players DB:")
    for first, last, pos, team in fb_league_players[:10]:  # Show first 10
        print(f"   - {first} {last} ({team})")
    if len(fb_league_players) > 10:
        print(f"   ... and {len(fb_league_players) - 10} more")
else:
    print("\nℹ️  No FB players found in NFL Players DB")

# Convert FB to RB in PROJECTIONS
cursor_proj.execute("""
    UPDATE projections 
    SET position = 'RB', updated_at = CURRENT_TIMESTAMP 
    WHERE position = 'FB'
""")
fb_proj_count = cursor_proj.rowcount
conn_proj.commit()

# Convert FB to RB in NFL PLAYERS
cursor_league.execute("""
    UPDATE nfl_players 
    SET position = 'RB', updated_at = CURRENT_TIMESTAMP 
    WHERE position = 'FB'
""")
fb_league_count = cursor_league.rowcount
conn_league.commit()

print(f"\n✅ Changed FB → RB:")
print(f"   Projections DB: {fb_proj_count} records updated")
print(f"   NFL Players DB: {fb_league_count} records updated")


🏃 Found 4 FB players in Projections DB:
   - Alec Ingold (MIA) from sleeper.com
   - Kyle Juszczyk (SF) from sleeper.com
   - C.J. Ham (MIN) from sleeper.com
   - Patrick Ricard (BAL) from sleeper.com

ℹ️  No FB players found in NFL Players DB

✅ Changed FB → RB:
   Projections DB: 4 records updated
   NFL Players DB: 0 records updated


## 7. Remove Defensive Players (DB, LB, DL, NT)


In [8]:
# Show defensive players before deletion in PROJECTIONS
cursor_proj.execute("""
    SELECT position, COUNT(*) 
    FROM projections 
    WHERE position IN ('DB', 'LB', 'DL', 'NT')
    GROUP BY position
""")
defensive_proj_breakdown = cursor_proj.fetchall()

if defensive_proj_breakdown:
    print("🛡️  Defensive players to remove from Projections DB:")
    for pos, count in defensive_proj_breakdown:
        print(f"   {pos}: {count} records")
else:
    print("ℹ️  No defensive players found in Projections DB")

# Show defensive players in NFL PLAYERS
cursor_league.execute("""
    SELECT position, COUNT(*) 
    FROM nfl_players 
    WHERE position IN ('DB', 'LB', 'DL', 'NT')
    GROUP BY position
""")
defensive_league_breakdown = cursor_league.fetchall()

if defensive_league_breakdown:
    print("\n🛡️  Defensive players to remove from NFL Players DB:")
    for pos, count in defensive_league_breakdown:
        print(f"   {pos}: {count} records")
else:
    print("\nℹ️  No defensive players found in NFL Players DB")

# Delete defensive players from PROJECTIONS
cursor_proj.execute("""
    DELETE FROM projections 
    WHERE position IN ('DB', 'LB', 'DL', 'NT')
""")
defensive_proj_count = cursor_proj.rowcount
conn_proj.commit()

# Delete defensive players from NFL PLAYERS
cursor_league.execute("""
    DELETE FROM nfl_players 
    WHERE position IN ('DB', 'LB', 'DL', 'NT')
""")
defensive_league_count = cursor_league.rowcount
conn_league.commit()

print(f"\n✅ Removed defensive players:")
print(f"   Projections DB: {defensive_proj_count} records deleted")
print(f"   NFL Players DB: {defensive_league_count} records deleted")


🛡️  Defensive players to remove from Projections DB:
   DL: 50 records
   NT: 2 records

ℹ️  No defensive players found in NFL Players DB

✅ Removed defensive players:
   Projections DB: 52 records deleted
   NFL Players DB: 0 records deleted


## 8. Fix Travis Hunter Position (JAX) → WR


In [9]:
# Check if Travis Hunter exists in PROJECTIONS
cursor_proj.execute("""
    SELECT player_first_name, player_last_name, position, team, source_website, projected_points 
    FROM projections 
    WHERE player_first_name = 'Travis' AND player_last_name = 'Hunter' AND team = 'JAX'
""")
travis_proj_records = cursor_proj.fetchall()

if travis_proj_records:
    print(f"🦁 Found Travis Hunter (JAX) in Projections DB - {len(travis_proj_records)} record(s):")
    for first, last, pos, team, source, points in travis_proj_records:
        print(f"   - {first} {last} | Position: {pos} | Team: {team} | Source: {source} | Points: {points}")
else:
    print("ℹ️  Travis Hunter (JAX) not found in Projections DB")

# Check if Travis Hunter exists in NFL PLAYERS
cursor_league.execute("""
    SELECT first_name, last_name, position, team, full_name
    FROM nfl_players 
    WHERE first_name = 'Travis' AND last_name = 'Hunter' AND team = 'JAX'
""")
travis_league_records = cursor_league.fetchall()

if travis_league_records:
    print(f"\n🦁 Found Travis Hunter (JAX) in NFL Players DB - {len(travis_league_records)} record(s):")
    for first, last, pos, team, full_name in travis_league_records:
        print(f"   - {full_name} | Position: {pos} | Team: {team}")
else:
    print("\nℹ️  Travis Hunter (JAX) not found in NFL Players DB")

# Update Travis Hunter's position to WR in PROJECTIONS
cursor_proj.execute("""
    UPDATE projections 
    SET position = 'WR', updated_at = CURRENT_TIMESTAMP 
    WHERE player_first_name = 'Travis' 
    AND player_last_name = 'Hunter' 
    AND team = 'JAX'
""")
hunter_proj_count = cursor_proj.rowcount
conn_proj.commit()

# Update Travis Hunter's position to WR in NFL PLAYERS
cursor_league.execute("""
    UPDATE nfl_players 
    SET position = 'WR', updated_at = CURRENT_TIMESTAMP 
    WHERE first_name = 'Travis' 
    AND last_name = 'Hunter' 
    AND team = 'JAX'
""")
hunter_league_count = cursor_league.rowcount
conn_league.commit()

print(f"\n✅ Updated Travis Hunter position to WR:")
print(f"   Projections DB: {hunter_proj_count} records updated")
print(f"   NFL Players DB: {hunter_league_count} records updated")


🦁 Found Travis Hunter (JAX) in Projections DB - 1 record(s):
   - Travis Hunter | Position: WR | Team: JAX | Source: fanduel.com | Points: 0.0

🦁 Found Travis Hunter (JAX) in NFL Players DB - 1 record(s):
   - Travis Hunter | Position: WR | Team: JAX

✅ Updated Travis Hunter position to WR:
   Projections DB: 1 records updated
   NFL Players DB: 1 records updated


## 9. Summary & Final State


In [10]:
# Get final counts for PROJECTIONS
cursor_proj.execute("SELECT COUNT(*) FROM projections")
total_proj_after = cursor_proj.fetchone()[0]

# Get final counts for NFL PLAYERS
cursor_league.execute("SELECT COUNT(*) FROM nfl_players")
total_league_after = cursor_league.fetchone()[0]

print("="*80)
print("📊 PROCESSING SUMMARY")
print("="*80)

print("\n🎯 PROJECTIONS DATABASE")
print("-" * 80)
print(f"Records before: {total_proj_before}")
print(f"Records after:  {total_proj_after}")
print(f"Records removed: {total_proj_before - total_proj_after}")
print()
print("Changes made:")
print(f"  ✅ WSH → WAS: {wsh_proj_count} records")
print(f"  ✅ JAC → JAX: {jac_proj_count} records")
print(f"  ✅ LA → LAR: {la_proj_count} records")
print(f"  ✅ D/ST → DST: {dst_proj_count} records")
print(f"  ✅ FB → RB: {fb_proj_count} records")
print(f"  ✅ Defensive players removed: {defensive_proj_count} records")
print(f"  ✅ Travis Hunter → WR: {hunter_proj_count} records")

print("\n🏈 SLEEPER NFL PLAYERS DATABASE")
print("-" * 80)
print(f"Records before: {total_league_before}")
print(f"Records after:  {total_league_after}")
print(f"Records removed: {total_league_before - total_league_after}")
print()
print("Changes made:")
print(f"  ✅ WSH → WAS: {wsh_league_count} records")
print(f"  ✅ JAC → JAX: {jac_league_count} records")
print(f"  ✅ LA → LAR: {la_league_count} records")
print(f"  ✅ DEF → DST: {def_league_count} records")
print(f"  ✅ FB → RB: {fb_league_count} records")
print(f"  ✅ Defensive players removed: {defensive_league_count} records")
print(f"  ✅ Travis Hunter → WR: {hunter_league_count} records")

print("\n" + "="*80)

# Show updated team names in PROJECTIONS
cursor_proj.execute("SELECT DISTINCT team FROM projections WHERE team IS NOT NULL ORDER BY team")
teams_proj_after = [row[0] for row in cursor_proj.fetchall()]
print(f"\n🏈 Teams in Projections DB after processing ({len(teams_proj_after)}):")
print(f"   {', '.join(teams_proj_after)}")

# Show updated positions in PROJECTIONS
cursor_proj.execute("SELECT position, COUNT(*) FROM projections GROUP BY position ORDER BY COUNT(*) DESC")
positions_proj_after = cursor_proj.fetchall()
print("\n📋 Positions in Projections DB after processing:")
for pos, count in positions_proj_after:
    print(f"   {pos}: {count}")

# Show updated team names in NFL PLAYERS
cursor_league.execute("SELECT DISTINCT team FROM nfl_players WHERE team IS NOT NULL ORDER BY team")
teams_league_after = [row[0] for row in cursor_league.fetchall()]
print(f"\n🏈 Teams in NFL Players DB after processing ({len(teams_league_after)}):")
print(f"   {', '.join(teams_league_after)}")

# Show updated positions in NFL PLAYERS
cursor_league.execute("SELECT position, COUNT(*) FROM nfl_players GROUP BY position ORDER BY COUNT(*) DESC")
positions_league_after = cursor_league.fetchall()
print("\n📋 Positions in NFL Players DB after processing:")
for pos, count in positions_league_after[:15]:  # Show top 15
    print(f"   {pos}: {count}")

# Close connections
conn_proj.close()
conn_league.close()

print("\n" + "="*80)
print("✨ Processing complete! Both databases are now clean and standardized.")
print("="*80)


📊 PROCESSING SUMMARY

🎯 PROJECTIONS DATABASE
--------------------------------------------------------------------------------
Records before: 1313
Records after:  1261
Records removed: 52

Changes made:
  ✅ WSH → WAS: 26 records
  ✅ JAC → JAX: 19 records
  ✅ LA → LAR: 15 records
  ✅ D/ST → DST: 28 records
  ✅ FB → RB: 4 records
  ✅ Defensive players removed: 52 records
  ✅ Travis Hunter → WR: 1 records

🏈 SLEEPER NFL PLAYERS DATABASE
--------------------------------------------------------------------------------
Records before: 3968
Records after:  3968
Records removed: 0

Changes made:
  ✅ WSH → WAS: 0 records
  ✅ JAC → JAX: 0 records
  ✅ LA → LAR: 0 records
  ✅ DEF → DST: 0 records
  ✅ FB → RB: 0 records
  ✅ Defensive players removed: 0 records
  ✅ Travis Hunter → WR: 1 records


🏈 Teams in Projections DB after processing (29):
   ARI, ATL, BAL, BUF, CAR, CHI, CLE, DEN, DET, FA, GB, HOU, IND, JAX, LAC, LAR, LV, MIA, MIN, NE, NO, NYG, NYJ, PHI, PIT, SEA, SF, TB, WAS

📋 Positions in P

## 10. Optional: Export Cleaned Data


In [ ]:
# Optional: Export cleaned data to CSV for review

# PROJECTIONS DATABASE
conn_proj = sqlite3.connect('../data/databases/projections.db')
df_proj = pd.read_sql_query("SELECT * FROM projections ORDER BY projected_points DESC", conn_proj)
conn_proj.close()

print("📊 PROJECTIONS DATABASE")
print("=" * 80)
print(f"Total records: {len(df_proj)}")
print(f"Columns: {list(df_proj.columns)}")
print(f"\nTop 10 projections:")
display(df_proj.head(10))

# NFL PLAYERS DATABASE
conn_league = sqlite3.connect('../data/databases/league.db')
df_league = pd.read_sql_query("SELECT * FROM nfl_players ORDER BY search_rank LIMIT 1000", conn_league)
conn_league.close()

print("\n📊 NFL PLAYERS DATABASE (top 1000 by search rank)")
print("=" * 80)
print(f"Total records retrieved: {len(df_league)}")
print(f"Columns: {list(df_league.columns)}")
print(f"\nTop 10 NFL players:")
display(df_league[['full_name', 'position', 'team', 'status', 'search_rank']].head(10))

# Uncomment to export:
df_proj.to_csv('../data/csv/cleaned_projections.csv', index=False)
df_league.to_csv('../data/csv/cleaned_nfl_players.csv', index=False)
print("\n💾 Exported to cleaned_projections.csv and cleaned_nfl_players.csv")


📊 PROJECTIONS DATABASE
Total records: 1261
Columns: ['id', 'source_website', 'week', 'player_first_name', 'player_last_name', 'position', 'projected_points', 'created_at', 'updated_at', 'team']

Top 10 projections:


,id,source_website,week,player_first_name,player_last_name,position,projected_points,created_at,updated_at,team
0,11346,sleeper.com,Week 10,Josh,Allen,QB,25.6,2025-11-05 04:12:37,2025-11-05 04:12:37,BUF
1,11347,sleeper.com,Week 10,Lamar,Jackson,QB,25.1,2025-11-05 04:12:37,2025-11-05 04:12:37,BAL
2,12010,fantasypros.com,Week 10,Josh,Allen,QB,25.0,2025-11-05 04:14:45,2025-11-05 04:14:45,BUF
3,12011,fantasypros.com,Week 10,Lamar,Jackson,QB,24.5,2025-11-05 04:14:45,2025-11-05 04:14:45,BAL
4,11804,espn.com,Week 10,Christian,McCaffrey,RB,24.4,2025-11-05 04:13:34,2025-11-05 04:13:34,SF
5,11348,sleeper.com,Week 10,Jaxson,Dart,QB,24.1,2025-11-05 04:12:37,2025-11-05 04:12:37,NYG
6,12012,fantasypros.com,Week 10,Drake,Maye,QB,24.0,2025-11-05 04:14:45,2025-11-05 04:14:45,NE
7,11349,sleeper.com,Week 10,Christian,McCaffrey,RB,23.8,2025-11-05 04:12:37,2025-11-05 04:12:37,SF
8,11350,sleeper.com,Week 10,Puka,Nacua,WR,23.7,2025-11-05 04:12:37,2025-11-05 04:12:37,LAR
9,11775,espn.com,Week 10,Lamar,Jackson,QB,23.6,2025-11-05 04:13:34,2025-11-05 04:13:34,BAL



📊 NFL PLAYERS DATABASE (top 1000 by search rank)
Total records retrieved: 1000
Columns: ['player_id', 'full_name', 'first_name', 'last_name', 'position', 'team', 'number', 'age', 'height', 'weight', 'college', 'years_exp', 'birth_date', 'birth_city', 'birth_state', 'birth_country', 'high_school', 'status', 'active', 'injury_status', 'injury_body_part', 'injury_notes', 'injury_start_date', 'practice_participation', 'depth_chart_position', 'depth_chart_order', 'search_rank', 'fantasy_positions', 'metadata', 'created_at', 'updated_at']

Top 10 NFL players:


,full_name,position,team,status,search_rank
0,Matt Seybert,TE,None,Active,NaN
1,Player Invalid,TE,None,Inactive,NaN
2,Player Invalid,RB,None,Inactive,NaN
3,Jamie Newman,QB,None,Inactive,NaN
4,Darius Clark,RB,None,Active,NaN
5,Player Invalid,RB,None,Inactive,NaN
6,HOU Defense,DST,HOU,None,NaN
7,Player Invalid,WR,None,Inactive,NaN
8,NE Defense,DST,NE,None,NaN
9,Bru McCoy,WR,None,Active,NaN



💾 Exported to cleaned_projections.csv and cleaned_nfl_players.csv
